In [ ]:
import os
os.chdir('C:\\Users\\linhanjie\\Desktop\\hzcj\\')
import pandas as pd
import numpy as np
from dateutil.parser import parse
from SelfUseModule.extract_module import order_history_extract
from SelfUseModule.mysql_conn_hzcj import connect_mysql_hzcj, connect_mysql_test
from SelfUseModule.mongodb_conn import connect_mongo
from _34_开单续费._3_整体判断汇总 import product_dict
from _1_后端资源查找资源意向 import seek_resource_intention, resource_intention_dict
from SelfUseModule2.wxzs_wechat import get_wxzs_qw_id


def get_resource_qd(st, en, conn):
    sql = """
    SELECT *
    FROM channel_resource_data_front
    WHERE local_create_time>='{st}' 
    AND local_create_time<ADDDATE('{en}',INTERVAL 1 DAY)
    """
    df = pd.read_sql(sql.format(st=st, en=en), conn)
    return df
def get_resource_hd(st, en, conn):
    sql = """
    SELECT *
    FROM channel_resource_data_back
    WHERE local_create_time>='{st}' 
    AND local_create_time<ADDDATE('{en}',INTERVAL 1 DAY)
    """
    df = pd.read_sql(sql.format(st=st, en=en), conn)
    return df
def if_order_flag(r_time, o_time, m=2):
    m_cha = m - 1
    r_time_y = r_time.year
    r_time_m = r_time.month
    o_time_y = o_time.year
    o_time_m = o_time.month

    m_num = (o_time_y - r_time_y) * 12 + o_time_m - r_time_m

    if o_time >= r_time and m_num <= m_cha:
        order_flag = 1
    else:
        order_flag = 0
    return order_flag

In [ ]:
st='2024-01-01'
en='2024-05-30'

conn_hzcj = connect_mysql_hzcj()
conn_test = connect_mysql_test()
db = connect_mongo()
coll = db.mg_CirculationHistory
# 资源数据
df_resource_qd = get_resource_qd('2024-01-01', en, conn_hzcj)
df_resource_qd['qd_local_create_time'] = df_resource_qd['local_create_time']
df_resource_qd['qd_id'] = df_resource_qd.apply(
    lambda x: get_wxzs_qw_id(x._id, conn_test) if x.leadInStyle == 'normal_wechat' else x._id, axis=1
)

df_resource_hd = get_resource_hd(st, en, conn_hzcj)
df_resource_hd = df_resource_hd[~df_resource_hd['back_group'].str.contains('管家')]
df_resource_hd['group'] = df_resource_hd['back_group'].apply(lambda x: '升级' + str(x).split('升级')[1] + str(x).split('升级')[2] if '管家' not in x else x)
df_resource_hd = pd.merge(df_resource_hd, df_resource_qd[['qd_id', 'cid', 'qd_local_create_time']], on='cid', how='inner')
df_resource_hd['dt_flag'] = df_resource_hd.apply(lambda x: 1 if x.local_create_time > x.qd_local_create_time else 0, axis=1)
df_resource_hd = df_resource_hd[df_resource_hd['dt_flag'] == 1]
df_resource_hd.sort_values(['_id', 'qd_local_create_time'], ascending=True, inplace=True, ignore_index=True)
df_resource_hd = pd.DataFrame(df_resource_hd.groupby(['_id']).last())
df_resource_hd.reset_index(drop=False, inplace=True)
df_resource_hd['resource_intention'] = df_resource_hd['qd_id'].apply(lambda x: seek_resource_intention(x, coll))
df_resource_hd = df_resource_hd[~df_resource_hd['resource_intention'].isnull()]

In [ ]:
# 开单数据
df_order = order_history_extract('2024-03-01', '2024-05-30')
# df_order['group'] = df_order['organization_name']
df_order_hd = df_order[(df_order['organization_name'].str.contains('升级')) &
                       (~df_order['organization_name'].str.contains('管家')) &
                       (df_order['group_name'].isin(product_dict['后端']))]
# 资源匹配开单
df_ro = pd.merge(df_resource_hd, df_order_hd, on=['cid'], how='left')
df_ro['order_flag'] = df_ro.apply(lambda x: if_order_flag(x.local_create_time, x.local_belong_date), axis=1)

In [ ]:
df_ro[df_ro['order_flag'] == 1].to_csv(r'C:\Users\linhanjie\Desktop\test\530\df_ro.csv')

In [ ]:
df_kd=df_ro[df_ro['order_flag'] == 1]

In [ ]:
df_kd['local_belong_date'] = pd.to_datetime(df_kd['local_belong_date'])
df_kd['month'] = df_kd['local_belong_date'].dt.to_period('M')

In [ ]:
# 按照新创建的'month'字段进行分组
grouped = df_kd.groupby('month')

# 现在grouped是一个GroupBy对象，你可以对它进行迭代或者应用聚合函数
# 例如，打印每个月的分组
dfzz = pd.DataFrame()
for month, group in grouped:
    print(f"Month: {month}")
    gg=group.groupby('resource_intention')
    gg=group.groupby(['resource_intention','name'])['actually_paid_y'].sum().reset_index()
    gg_sort =gg.sort_values(by=['resource_intention', 'actually_paid_y'], ascending=[True, False])
    # 步骤3: 计算每个小组的金额总和
    grouped_totals = gg_sort.groupby('resource_intention')['actually_paid_y'].sum().reset_index()
    
    # 步骤4: 对每个小组中的名字按金额排序，并取前30%的名字
    top_30_percent_df = pd.DataFrame()
    for group, group_data in gg_sort.groupby('resource_intention'):
        # 计算前30%的阈值
        total_amount = grouped_totals[grouped_totals['resource_intention']==group]['actually_paid_y'].iloc[0]
        threshold = total_amount * 0.3
        cumulative_sum = group_data['actually_paid_y'].cumsum()
        mask = cumulative_sum <= threshold
        last_index_in_threshold = mask.sum() - 1
        # 步骤4.5: 如果临界点的金额有多个名字相同，将这些名字全部包括在内
        current_group = group_data[group_data['resource_intention'] == group].reset_index()
        same_amount_mask = (current_group['actually_paid_y'] == current_group['actually_paid_y'].iloc[last_index_in_threshold])
        include_all_same_amount = current_group[same_amount_mask | (current_group.index <= last_index_in_threshold)]
        include_all_same_amount = include_all_same_amount.drop(columns='index') 
        top_30_percent_df=pd.concat([top_30_percent_df,include_all_same_amount])
    top_30_percent_df['month']=month
    dfzz=pd.concat([dfzz,top_30_percent_df])
dfzz.columns=['意向','姓名','开单额','月份']
dfzz=pd.merge(dfzz,df_kd[['name','organization_name']].drop_duplicates(),left_on='姓名',right_on='name',how='left')[['意向','姓名','开单额','月份','organization_name']]
dfzz.columns=['意向','姓名','开单额','月份','小组']

In [ ]:
group30 = dfzz.groupby('意向')
lianxu=pd.DataFrame()
for res, group in group30:
    name_counts=group['姓名'].value_counts()
    names_more_than_three = name_counts[name_counts > 2].index.tolist()
    filtered_df = group[group['姓名'].isin(names_more_than_three)].sort_values(by='姓名')
    lianxu=pd.concat([lianxu,filtered_df])

In [ ]:
# 按照新创建的'month'字段进行分组
grouped = df_kd.groupby('month')

# 现在grouped是一个GroupBy对象，你可以对它进行迭代或者应用聚合函数
# 例如，打印每个月的分组
dfgzz = pd.DataFrame()
for month, group in grouped:
    print(f"Month: {month}")
    gg=group.groupby('resource_intention')
    gg=group.groupby(['resource_intention','organization_name'])['actually_paid_y'].sum().reset_index()
    gg_sort =gg.sort_values(by=['resource_intention', 'actually_paid_y'], ascending=[True, False])
    top5_df = pd.DataFrame()
    for group, group_data in gg_sort.groupby('resource_intention'):
        cc= group_data.reset_index().drop(columns='index') 
        if len(cc) >4:
            top5 = cc[cc['actually_paid_y']>=cc.iloc[4,2]]
        else:
            top5 = cc[cc['actually_paid_y']>=cc.iloc[len(cc)-1,2]]
        top5_df=pd.concat([top5_df,top5])
    top5_df['month']=month
    dfgzz=pd.concat([dfgzz,top5_df])
dfgzz.columns=['意向','小组','开单额','月份']

In [ ]:
group5 = dfgzz.groupby('意向')
lianxugroup=pd.DataFrame()
for res, group in group5:
    name_counts=group['小组'].value_counts()
    names_more_than_three = name_counts[name_counts > 2].index.tolist()
    filtered_df = group[group['小组'].isin(names_more_than_three)].sort_values(by='小组')
    lianxugroup=pd.concat([lianxugroup,filtered_df])

In [ ]:
# dfzz.to_excel(r'C:\Users\linhanjie\Desktop\test\530\dfzz.xlsx')
with pd.ExcelWriter(r'C:\Users\linhanjie\Desktop\test\530\意向开单前排.xlsx') as writer:
    dfzz.to_excel(writer, sheet_name='每个月在前30%员工', index=False)
    lianxu.to_excel(writer, sheet_name='连续三个月在前30%员工', index=False)
    dfgzz.to_excel(writer, sheet_name='每个月在前五小组', index=False)
    lianxugroup.to_excel(writer, sheet_name='连续三个月在前五小组', index=False)
    # df_back_w_num_month_all.to_excel(writer, sheet_name='Sheet2', index=False)

In [ ]:
dfzz

In [ ]:
same_amount_mask

In [ ]:
current_group